In [13]:
import random
from six.moves import urllib

In [14]:
FLOWERS_DIR = './flower_photos'
TRAIN_FRACTION = 0.8
RANDOM_SEED = 2018
def download_images():
  """If the images aren't already downloaded, save them to FLOWERS_DIR."""
  if not os.path.exists(FLOWERS_DIR):
    DOWNLOAD_URL = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
    print('Downloading flower images from %s...' % DOWNLOAD_URL)
    urllib.request.urlretrieve(DOWNLOAD_URL, 'flower_photos.tgz')
    !tar xfz flower_photos.tgz
  print('Flower photos are located in %s' % FLOWERS_DIR)

In [15]:
download_images()

Flower photos are located in ./flower_photos


In [16]:
from keras.applications.inception_v3 import InceptionV3 
from keras.preprocessing import image
from keras.models import Model, load_model 
from keras.callbacks import ModelCheckpoint 
from keras.layers import Dense, GlobalAveragePooling2D 
from keras.preprocessing.image import ImageDataGenerator 
from keras.optimizers import SGD 
import os. path
MODEL_FILE = "flowers. hd5"

In [17]:
# Create a model if none exists. Freezes all training except in
# newly attached output layers. We can specify the number of nodes
# in the hidden penultimate layer, and the number of output
# categories.
def create_model(num_hidden, num_classes):
    base_model = InceptionV3 (include_top= False, weights ='imagenet')
    x = base_model.output
    x = GlobalAveragePooling2D() (x)
    X = Dense(num_hidden, activation='relu') (x)
    predictions = Dense(num_classes, activation='softmax') (x)
    

    
    for layer in base_model.layers:
        layer.trainable = False
    
    model = Model(inputs=base_model.input, outputs = predictions)
    return model

In [18]:
x = base_model.output 
x = GlobalAveragePooling2D() (x) 
x = Dense(num_hidden, activation='relu') (x) 
predictions = Dense(num_classes, activation='softmax') (x)

NameError: name 'base_model' is not defined

In [19]:
for layer in base_model.layers:
    layer.trainable = False

NameError: name 'base_model' is not defined

In [ ]:
model = Model(inputs=base_model.imput ,outputs = predictions)
return model

In [ ]:
# Loads an existing model file, then sets only the last 
# # 3 layers (which we added) to trainable.
def load_existing (model_file):
    # Load the model 
    model = load_model(model_file)
    # Set only last 3 layers as trainable 
    numlayers = len (model.layers),
    for layer in model.layers[:numlayers-3]:
        layer.trainable = False
    for layer in model.layers [numlayers-3:]:
        layer.trainable = True
    return model